In [1]:
!nvidia-smi

Fri Dec 11 00:58:08 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [3]:
!pip3 install gast==0.2.2
!pip3 list | grep gast

  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7542 sha256=5714323878aa6ff1857c4975b6d900d7b293934f32e70faba5bbb07b23d5ab7d
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
gast                          0.2.2          


In [4]:
import tensorflow.compat.v1 as tf
print(tf.__version__)
print(tf.test.is_gpu_available())
import keras
print(keras.__version__)

1.15.2
True
2.3.1


Using TensorFlow backend.


In [5]:
import os
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"

# AMP要使用 tf.keras 
os.environ["TF_KERAS"] = "1"

In [6]:
# coding=utf-8

import os
from distutils.util import strtobool

# 判断是tf.keras还是纯keras的标记
is_tf_keras = strtobool(os.environ.get('TF_KERAS', '0'))

if is_tf_keras:
    from tensorflow.keras import backend as K
else:
    from keras import backend as K

import numpy as np
import imageio


# 采样函数
def sample(path, g_model, img_dim, z_dim, n=9, z_samples=None):
    figure = np.zeros((img_dim * n, img_dim * n, 3))
    if z_samples is None:
        z_samples = np.random.randn(n**2, z_dim)
    for i in range(n):
        for j in range(n):
            z_sample = z_samples[[i * n + j]]
            x_sample = g_model.predict(z_sample)
            digit = x_sample[0]
            figure[i * img_dim:(i + 1) * img_dim,
                   j * img_dim:(j + 1) * img_dim] = digit
    figure = (figure + 1) / 2 * 255
    figure = np.round(figure, 0).astype(np.uint8)
    imageio.imwrite(path, figure)


# 谱归一化
class SpectralNormalization:
    """层的一个包装，用来加上SN。
    """

    def __init__(self, layer):
        self.layer = layer

    def spectral_norm(self, w, r=5):
        w_shape = K.int_shape(w)
        in_dim = np.prod(w_shape[:-1]).astype(int)
        out_dim = w_shape[-1]
        w = K.reshape(w, (in_dim, out_dim))
        u = K.ones((1, in_dim))
        for i in range(r):
            v = K.l2_normalize(K.dot(u, w))
            u = K.l2_normalize(K.dot(v, K.transpose(w)))
        return K.sum(K.dot(K.dot(u, w), K.transpose(v)))

    def spectral_normalization(self, w):
        return w / self.spectral_norm(w)

    def __call__(self, inputs):
        with K.name_scope(self.layer.name):
            if not self.layer.built:
                input_shape = K.int_shape(inputs)
                self.layer.build(input_shape)
                self.layer.built = True
                if self.layer._initial_weights is not None:
                    self.layer.set_weights(self.layer._initial_weights)
        if not hasattr(self.layer, 'spectral_normalization'):
            if hasattr(self.layer, 'kernel'):
                self.layer.kernel = self.spectral_normalization(self.layer.kernel)
            if hasattr(self.layer, 'gamma'):
                self.layer.gamma = self.spectral_normalization(self.layer.gamma)
            self.layer.spectral_normalization = True
        return self.layer(inputs)


class ExponentialMovingAverage:
    """对模型权重进行指数滑动平均。
    用法：在model.compile之后、第一次训练之前使用；
    先初始化对象，然后执行inject方法。

    训练：
    EMAer = ExponentialMovingAverage(model) # 在模型compile之后执行
    EMAer.initialize() # 在模型compile之后执行
    EMAer.ema_on_batch() # 每个batch完成后执行

    预测：
    EMAer.apply_ema_weights() # 将EMA的权重应用到模型中
    model.predict(x_test) # 进行预测、验证、保存等操作

    继续训练：
    EMAer.reset_old_weights() # 继续训练之前，要恢复模型旧权重。还是那句话，EMA不影响模型的优化轨迹。
    """
    def __init__(self, model, momentum=0.999):
        self.momentum = momentum
        self.model = model
    def inject(self):
        """与旧代码兼容
        """
        self.initialize()
    def initialize(self):
        """ema_weights初始化跟原模型初始化一致。
        """
        self.old_weights = K.batch_get_value(self.model.weights)
        self.mv_trainable_weights_vals = {x.name: K.get_value(x) for x in
                                          self.model.trainable_weights}
    def apply_ema_weights(self):
        """备份原模型权重，然后将平均权重应用到模型上去。
        """
        self.old_weights = K.batch_get_value(self.model.weights)
        for weight in self.model.trainable_weights:
             K.set_value(weight, self.mv_trainable_weights_vals[weight.name])
    def reset_old_weights(self):
        """恢复模型到旧权重。
        """
        K.batch_set_value(zip(self.model.weights, self.old_weights))
    def ema_on_batch(self):
        for weight in self.model.trainable_weights:
            old_val = self.mv_trainable_weights_vals[weight.name]
            self.mv_trainable_weights_vals[weight.name] -= \
                (1.0 - self.momentum) * (old_val - K.get_value(weight))


class ExponentialMovingAverage_NEED_Keras_patch:
    """对模型权重进行指数滑动平均。
    用法：在model.compile之后、第一次训练之前使用；
    先初始化对象，然后执行inject方法。

    训练：
    EMAer = ExponentialMovingAverage(model) # 在模型compile之后执行
    EMAer.inject() # 在模型compile之后执行
    model.fit(x_train, y_train) # 训练模型

    预测：
    EMAer.apply_ema_weights() # 将EMA的权重应用到模型中
    model.predict(x_test) # 进行预测、验证、保存等操作

    继续训练：
    EMAer.reset_old_weights() # 继续训练之前，要恢复模型旧权重。还是那句话，EMA不影响模型的优化轨迹。
    model.fit(x_train, y_train) # 继续训练
    """
    '''
    权重滑动平均 EMA （需要给Keras 2.3.1打个补丁，才能生效） 
    diff --git a/keras/engine/training.py b/keras/engine/training.py
    index 0a556f21..1a9a374e 100644
    --- a/keras/engine/training.py
    +++ b/keras/engine/training.py
    @@ -328,7 +328,7 @@ class Model(Network):
                     self.train_function = K.function(
                         inputs,
                         [self.total_loss] + metrics_tensors,
    -                    updates=updates + metrics_updates,
    +                    updates=updates + metrics_updates + (self._other_metrics if hasattr(self, '_other_metrics') else []),
                         name='train_function',
                         **self._function_kwargs)
    '''
    def __init__(self, model, momentum=0.999):
        self.momentum = momentum
        self.model = model
        self.ema_weights = [K.zeros(K.shape(w)) for w in model.weights]
    def inject(self):
        """添加更新算子到model.metrics_updates。 
        """
        self.initialize()
        for w1, w2 in zip(self.ema_weights, self.model.weights):
            op = K.moving_average_update(w1, w2, self.momentum)
            #self.model.metrics_updates.append(op) # 在 keras 2.2.4 有效
            if not hasattr(self.model, '_other_metrics'):
                self.model._other_metrics = []
            self.model._other_metrics.append(op)
    def initialize(self):
        """ema_weights初始化跟原模型初始化一致。
        """
        self.old_weights = K.batch_get_value(self.model.weights)
        K.batch_set_value(zip(self.ema_weights, self.old_weights))
    def apply_ema_weights(self):
        """备份原模型权重，然后将平均权重应用到模型上去。
        """
        self.old_weights = K.batch_get_value(self.model.weights)
        ema_weights = K.batch_get_value(self.ema_weights)
        K.batch_set_value(zip(self.model.weights, ema_weights))
    def reset_old_weights(self):
        """恢复模型到旧权重。
        """
        K.batch_set_value(zip(self.model.weights, self.old_weights))


In [7]:
#! -*- coding: utf-8 -*-

import os
from distutils.util import strtobool

# 判断是tf.keras还是纯keras的标记
is_tf_keras = strtobool(os.environ.get('TF_KERAS', '0'))

if is_tf_keras:
    from tensorflow.keras.layers import *
    from tensorflow.keras.models import Model
    from tensorflow.keras import backend as K
else:
    from keras.layers import *
    from keras.models import Model
    from keras import backend as K

import numpy as np
#from utils import SpectralNormalization


class ScaleShift(Layer):
    """平移缩放
    """
    def __init__(self, **kwargs):
        super(ScaleShift, self).__init__(**kwargs)
    def call(self, inputs):
        z, beta, gamma = inputs
        for i in range(K.ndim(z) - 2):
            beta = K.expand_dims(beta, 1)
            gamma = K.expand_dims(gamma, 1)
        return z * (gamma + 1) + beta

# SELF-MODE
def SelfModulatedBatchNormalization(h, c, z_dim):
    num_hidden = z_dim
    dim = K.int_shape(h)[-1]
    h = BatchNormalization(center=False, scale=False)(h)
    beta = Dense(num_hidden, activation='relu')(c)
    beta = Dense(dim)(beta)
    gamma = Dense(num_hidden, activation='relu')(c)
    gamma = Dense(dim)(gamma)
    return ScaleShift()([h, beta, gamma])


# 生成器和判别器 模型
def load_model(img_dim, z_dim, activation=None, sn=False, use_bias=True, self_mode=False):
    num_layers = int(np.log2(img_dim)) - 3
    if img_dim > 256:
        max_num_channels = img_dim * 4
    else:    
        max_num_channels = img_dim * 8
    f_size = img_dim // 2**(num_layers + 1)

    # 谱归一化
    if sn:
        SN = SpectralNormalization
    else:
        SN = lambda xx: xx

    # 判别器
    x_in = Input(shape=(img_dim, img_dim, 3))
    x = x_in

    for i in range(num_layers + 1):
        num_channels = max_num_channels // 2**(num_layers - i)
        x = SN(Conv2D(num_channels,
                   (4, 4),
                   strides=(2, 2),
                   use_bias=use_bias,
                   padding='same'))(x)
        if i > 0:
            x = SN(BatchNormalization())(x)
        x = LeakyReLU(0.2)(x)

    x = Flatten()(x)
    x = SN(Dense(1, use_bias=use_bias, activation=activation))(x)

    d_model = Model(x_in, x)


    # 生成器
    z_in = Input(shape=(z_dim, ))
    z = z_in

    z = Dense(f_size**2 * max_num_channels)(z)
    z = Reshape((f_size, f_size, max_num_channels))(z)
    if self_mode: # SELF-MODE
        z = SelfModulatedBatchNormalization(z, z_in, z_dim)
    else:
        z = BatchNormalization()(z)
    z = Activation('relu')(z)

    for i in range(num_layers):
        num_channels = max_num_channels // 2**(i + 1)
        z = Conv2DTranspose(num_channels,
                            (4, 4),
                            strides=(2, 2),
                            padding='same')(z)
        if self_mode:  # SELF-MODE
            z = SelfModulatedBatchNormalization(z, z_in, z_dim)
        else:
            z = BatchNormalization()(z)
        z = Activation('relu')(z)

    z = Conv2DTranspose(3,
                        (4, 4),
                        strides=(2, 2),
                        padding='same')(z)
    z = Activation('tanh')(z)

    g_model = Model(z_in, z)

    return d_model, g_model



# 生成器和判别器 模型
def load_model_2(img_dim, z_dim, activation=None, sn=False, use_bias=True):
    num_layers = int(np.log2(img_dim)) - 3
    if img_dim > 256:
        max_num_channels = img_dim * 4
    else:    
        max_num_channels = img_dim * 8
    f_size = img_dim // 2**(num_layers + 1)

    # 谱归一化
    if sn:
        SN = SpectralNormalization
    else:
        SN = lambda xx: xx

    # 判别器
    x_in = Input(shape=(img_dim, img_dim, 3))
    x = x_in

    x = SN(Conv2D(128, 3))(x)
    x = LeakyReLU()(x)
    x = SN(Conv2D(128, 4, strides=2))(x)
    x = LeakyReLU()(x)
    x = SN(Conv2D(128, 4, strides=2))(x)
    x = LeakyReLU()(x)
    x = SN(Conv2D(128, 4, strides=2))(x)
    x = LeakyReLU()(x)
    x = Flatten()(x)
    x = Dropout(0.4)(x)
    x = SN(Dense(1, use_bias=use_bias, activation=activation))(x)

    d_model = Model(x_in, x)

    # 生成器
    z_in = Input(shape=(z_dim, ))
    z = z_in

    z = Dense(128 * (img_dim//2) * (img_dim//2))(z_in)
    z = LeakyReLU()(z)
    z = Reshape((img_dim//2, img_dim//2, 128))(z)
    z = Conv2D(128, 4, padding='same')(z)
    z = LeakyReLU()(z)
    z = Conv2DTranspose(128, 4, strides=2, padding='same')(z)
    z = LeakyReLU()(z)
    z = Conv2D(128, 4, padding='same')(z)
    z = LeakyReLU()(z)
    z = Conv2D(128, 4, padding='same')(z)
    z = LeakyReLU()(z)
    z = Conv2D(3, 7, activation='tanh', padding='same')(z)

    g_model = Model(z_in, z)


    return d_model, g_model


# 生成器和判别器 模型
def load_model_3(img_dim, z_dim, activation=None, sn=False, use_bias=True):
    num_layers = int(np.log2(img_dim)) - 3
    if img_dim > 256:
        max_num_channels = img_dim * 4
    else:    
        max_num_channels = img_dim * 8
    f_size = img_dim // 2**(num_layers + 1)

    # 谱归一化
    if sn:
        SN = SpectralNormalization
    else:
        SN = lambda xx: xx

    # 判别器
    x_in = Input(shape=(img_dim, img_dim, 3))
    x = x_in

    x = SN(Conv2D(64, 3, strides=2, padding='same'))(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Dropout(0.25)(x)
    x = SN(Conv2D(128, 3, strides=2, padding='same'))(x)
    x = ZeroPadding2D(padding=((0,1),(0,1)))(x)
    x = SN(BatchNormalization(momentum=0.8))(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Dropout(0.25)(x)
    x = SN(Conv2D(256, 3, strides=2, padding='same'))(x)
    x = SN(BatchNormalization(momentum=0.8))(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Dropout(0.25)(x)
    x = SN(Conv2D(512, 3, strides=2, padding='same'))(x)
    x = SN(BatchNormalization(momentum=0.8))(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Dropout(0.25)(x)
    x = Flatten()(x)
    x = SN(Dense(1, use_bias=use_bias, activation=activation))(x)

    d_model = Model(x_in, x)

    # 生成器
    z_in = Input(shape=(z_dim, ))
    z = z_in

    z = Dense(128 * (img_dim//4) * (img_dim//4))(z_in)
    z = LeakyReLU(alpha=0.2)(z)
    z = Reshape((img_dim//4, img_dim//4, 128))(z)
    z = UpSampling2D()(z)
    z = Conv2D(128, 4, padding='same')(z)
    z = BatchNormalization(momentum=0.8)(z)
    z = LeakyReLU(alpha=0.2)(z)
    z = UpSampling2D()(z)
    z = Conv2D(64, 4, padding='same')(z)
    z = BatchNormalization(momentum=0.8)(z)
    z = LeakyReLU(alpha=0.2)(z)
    z = Conv2D(3, 7, activation='tanh', padding='same')(z)

    g_model = Model(z_in, z)


    return d_model, g_model



In [25]:
#! -*- coding: utf-8 -*-
import os
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"

# AMP要使用 tf.keras 
os.environ["TF_KERAS"] = "1"

import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#from utils import sample, ExponentialMovingAverage
#from models import load_model


if not os.path.exists('samples'):
    os.mkdir('samples')


img_dim = 256
z_dim = 100
EMA = True # whether use EMA
L1_or_L2 = 'L1' #  L1 或 L2
total_iter = 1000000
batch_size = 128
iters_per_sample = 100 # 采样频率


img_dir = '/content/CASIA-maxpy-clean'

# 数据生成器
img_datagen = ImageDataGenerator(
    preprocessing_function=lambda x: x.astype(np.float32) / 255 * 2 - 1,
    zoom_range=0.0 # 缩放， 0.5 放大
)
img_generator = img_datagen.flow_from_directory(
    img_dir,
    target_size=(img_dim, img_dim),
    batch_size=batch_size,
    class_mode=None # 只生成图片，不生成标签
)


# 载入基本模型： 判别器，生成器
d_model, g_model = load_model(img_dim, z_dim, use_bias=False, self_mode=True)
d_model.summary()
g_model.summary()

# AMP 混合精度
opt = Adam(2e-4, 0.5)
opt = tf.train.experimental.enable_mixed_precision_graph_rewrite(opt)

# 整合模型（训练判别器）
x_in = Input(shape=(img_dim, img_dim, 3))
z_in = Input(shape=(z_dim, ))
g_model.trainable = False

x_real = x_in
x_fake = g_model(z_in)

x_real_score = d_model(x_real)
x_fake_score = d_model(x_fake)

d_train_model = Model([x_in, z_in],
                      [x_real_score, x_fake_score])

d_loss = x_real_score - x_fake_score
d_loss = d_loss[:, 0]
if L1_or_L2=='L1':
    d_norm = 10 * K.mean(K.abs(x_real - x_fake), axis=[1, 2, 3])
else:
    d_norm = 10 * K.sqrt(K.mean(K.square(x_real - x_fake), axis=[1, 2, 3]))
d_loss = K.mean(- d_loss + 0.5 * d_loss**2 / d_norm)

d_train_model.add_loss(d_loss)
d_train_model.compile(optimizer=opt)


# 整合模型（训练生成器）
g_model.trainable = True
d_model.trainable = False

x_real = x_in
x_fake = g_model(z_in)

x_real_score = d_model(x_real)
x_fake_score = d_model(x_fake)

g_train_model = Model([x_in, z_in],
                      [x_real_score, x_fake_score])

g_loss = K.mean(x_real_score - x_fake_score)

g_train_model.add_loss(g_loss)
g_train_model.compile(optimizer=opt)


# 检查模型结构
d_train_model.summary()
g_train_model.summary()


# 装入checkpoints， 只保存了生成模型的权重
g_train_model.load_weights('out/g_train_ema_model.weights')
g_model = g_train_model.layers[2]
d_model = g_train_model.layers[3]

# EMA
if EMA:
    EMAer_g_train = ExponentialMovingAverage(g_train_model, 0.999) # 在模型compile之后执行
    EMAer_g_train.inject() # 在模型compile之后执行


if __name__ == '__main__':


    n_size = 6
    Z = np.random.randn(n_size**2, z_dim)

    for i in range(total_iter):
        for j in range(2):
            x_sample = next(img_generator)
            z_sample = np.random.randn(len(x_sample), z_dim)
            d_loss = d_train_model.train_on_batch(
                [x_sample, z_sample], None)
        for j in range(1):
            x_sample = next(img_generator)
            z_sample = np.random.randn(len(x_sample), z_dim)
            g_loss = g_train_model.train_on_batch(
                [x_sample, z_sample], None)
            if EMA:
                EMAer_g_train.ema_on_batch()
        if i % 10 == 0:
            print('iter: %s, d_loss: %s, g_loss: %s' % (i, d_loss, g_loss))
        if i % iters_per_sample == 0:
            sample('samples/test_%s.png' % i, g_model, img_dim, z_dim, n=n_size, z_samples=Z)
            g_train_model.save_weights('./g_train_model.weights')
            if EMA:
                EMAer_g_train.apply_ema_weights() # 将EMA的权重应用到模型中
                sample('samples/test_ema_%s.png' % i, g_model, img_dim, z_dim, n=n_size, z_samples=Z)
                g_train_model.save_weights('./g_train_ema_model.weights')
                EMAer_g_train.reset_old_weights() # 继续训练之前，要恢复模型旧权重


Found 2412 images belonging to 82 classes.
Model: "model_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_21 (InputLayer)        [(None, 256, 256, 3)]     0         
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 128, 128, 64)      3072      
_________________________________________________________________
leaky_re_lu_28 (LeakyReLU)   (None, 128, 128, 64)      0         
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 64, 64, 128)       131072    
_________________________________________________________________
batch_normalization_51 (Batc (None, 64, 64, 128)       512       
_________________________________________________________________
leaky_re_lu_29 (LeakyReLU)   (None, 64, 64, 128)       0         
_________________________________________________________________
conv2d_30 (Conv

KeyboardInterrupt: ignored

In [24]:
#!rm -rf /content/CASIA-maxpy-clean
#!tar xvfJ /content/drive/MyDrive/colab_data/data/CASIA-maxpy-clean.xv
!rm -rf /content/samples